## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2018-12-26 01:50:47--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2018-12-26 01:50:47--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd56848edf348e94abe0a88e1a.dl.dropboxusercontent.com/cd/0/get/AYIhDRUgKnMyhRY8FiwtyBesmvUGvKY1sVv_TIQvL-v3D_TixTFGqgHdh99KvsBOvnlKaCKPZQcTaOn56lE2yuEZvUZbcwLnb72v5E3uE2FjXPbNwU0krvZki8c_6AJCGVIXPq1n2Xe0GM0484IjxTHLI71wiz4jMM42PDY4196s05CFYuRZ8SQGVhnXnPVx5pI/file?dl=1 [following]
--2018-12-26 01:50:47--  https://uccd56848edf348e94abe0a88e1a.dl.dropboxusercontent.com/cd/0/get/AYIhDRUgKnMyhRY8FiwtyBesmvUGvKY1sVv_TIQvL-v3D_TixTFGqgHdh9

In [2]:
import numpy as np

data = list(open("./quora.txt"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [0]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = []
for i in range(len(data)):
    data_tok.append(tokenizer.tokenize(data[i].lower()))

In [0]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [11]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [0]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [13]:
# now you can get word vectors !
model.get_vector('anything')

array([ 4.255281  , -2.0810664 , -0.18408601,  0.79312205,  1.2066268 ,
       -0.07505123, -2.1990025 , -2.3366425 , -1.2857571 ,  0.60148156,
        2.2370756 ,  1.6339492 , -2.534834  , -2.5884252 ,  0.14263703,
       -1.045357  , -1.2628125 ,  1.3978221 ,  2.4959264 , -3.180694  ,
        2.32307   , -2.1628578 ,  0.21091232,  0.20874023,  2.7805555 ,
       -1.2443687 ,  4.4286957 ,  0.8122711 , -0.00869262, -0.41263184,
        3.019628  ,  1.5872115 ], dtype=float32)

In [14]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('rice', 0.9411166906356812),
 ('fruit', 0.9261350631713867),
 ('cheese', 0.9187461137771606),
 ('butter', 0.9145932197570801),
 ('sauce', 0.9099643230438232),
 ('corn', 0.9065866470336914),
 ('chicken', 0.9059572815895081),
 ('honey', 0.9057888984680176),
 ('soup', 0.903209924697876),
 ('chocolate', 0.8990206718444824)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [0]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [17]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.538511335849762),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233934879303),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [18]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [0]:
# for each word, compute it's vector with model
word_vectors = np.empty((0, 100))
for i in range(len(words)):
    word_vectors = np.vstack((word_vectors, model.get_vector(words[i])))

In [0]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [0]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2, random_state=1234)
word_vectors_pca = pca.fit_transform(word_vectors)
word_vectors_pca = scale(word_vectors_pca)

# and maybe MORE OF YOUR CODE here :)

In [0]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [24]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [25]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1003', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [26]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca

tsne = TSNE(n_components=2, verbose=100, random_state=1234)
word_tsne = tsne.fit_transform(word_vectors)
word_tsne = scale(word_tsne)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.003s...
[t-SNE] Computed neighbors for 1000 samples in 0.241s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.057s
[t-SNE] Iteration 50: error = 68.8735352, gradient norm = 0.3093952 (50 iterations in 2.038s)
[t-SNE] Iteration 100: error = 68.6248093, gradient norm = 0.3035527 (50 iterations in 1.622s)
[t-SNE] Iteration 150: error = 68.3412857, gradient norm = 0.2989291 (50 iterations in 1.599s)
[t-SNE] Iteration 200: error = 69.2266388, gradient norm = 0.2853951 (50 iterations in 1.571s)
[t-SNE] Iteration 250: error = 69.2950745, gradient norm = 0.2838839 (50 iterations in 1.658s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.295074
[t-SNE] Iteration 300: error = 1.2443764, gradient norm = 0.0024609 (50 iterations in 1.419s)
[t-SNE] Iteration 350: error = 1.1142710, gradient norm = 0.00066

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [27]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1112', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [0]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    phrase = tokenizer.tokenize(phrase.lower())
    num = 0
    
    for i in range(len(phrase)):
        if phrase[i] in model.vocab.keys():
            vector = vector + model.get_vector(phrase[i])
            num = num + 1
        
    vector = vector / num
    
    return vector
        
    

In [0]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [0]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
chosen_tok = []
for i in range(len(chosen_phrases)):
    chosen_tok.append(tokenizer.tokenize(chosen_phrases[i].lower()))

phrase_vectors = np.empty((0, 100))

for i in range(len(chosen_tok)):
    one_phrase_vectors = np.zeros([model.vector_size], dtype='float32')
    for j in range(len(chosen_tok[i])):
        if chosen_tok[i][j] in model.vocab.keys():
            one_phrase_vectors = one_phrase_vectors + model.get_vector(chosen_tok[i][j])
      
    phrase_vectors = np.vstack((phrase_vectors, one_phrase_vectors))



# # phrase_vectors = np.zeros([0, 100], dtype='float32')
# for i in range(len(chosen_tok)):
# #   print(chosen_tok[i])
#   if chosen_tok[i] in model.vocab.keys():
# #     phrase_vectors = phrase_vectors + model.get_vector(chosen_tok[i])
#     phrase_vectors = np.vstack((phrase_vectors, model.get_vector(chosen_tok[i])))

In [96]:
phrase_vectors.shape

(1001, 100)

In [0]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [98]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.004s...
[t-SNE] Computed neighbors for 1001 samples in 0.163s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 5.812442
[t-SNE] Computed conditional probabilities in 0.070s
[t-SNE] Iteration 50: error = 70.6550446, gradient norm = 0.3388840 (50 iterations in 2.057s)
[t-SNE] Iteration 100: error = 72.6174240, gradient norm = 0.3079655 (50 iterations in 1.588s)
[t-SNE] Iteration 150: error = 71.4708405, gradient norm = 0.2989936 (50 iterations in 1.640s)
[t-SNE] Iteration 200: error = 72.7868271, gradient norm = 0.2940517 (50 iterations in 1.567s)
[t-SNE] Iteration 250: error = 71.6650772, gradient norm = 0.2917077 (50 iterations in 1.608s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 71.665077
[t-SNE] Iteration 300: error = 1.3868402, gradient norm = 0.0043394 (50 iterations in 1.264s)

In [99]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1230', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [100]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [0]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_vector = get_phrase_embedding(query)
    
    distance = np.zeros([len(data_vectors)])
    for i in range(len(data_vectors)):
        distance[i] = np.dot(query_vector, data_vectors[i]) / (np.linalg.norm(query_vector) * np.linalg.norm(data_vectors[i]))
      
    most_k_similar_index = np.argsort(-distance)
    
    most_k_similar = []
    for i, j in enumerate(list(most_k_similar_index[0: k])):
        most_k_similar.append(data[j])
    
    
    return most_k_similar

In [195]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [192]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of India?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [193]:
find_nearest(query="Why don't i ask a question myself?", k=10)

["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.